#AutoEncoders

##Downloading the dataset

###ML-100K

In [5]:
!wget "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
!unzip ml-100k.zip
!ls

--2021-07-21 22:48:35--  http://files.grouplens.org/datasets/movielens/ml-100k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4924029 (4.7M) [application/zip]
Saving to: ‘ml-100k.zip’

ml-100k.zip         100%[===================>]   4.70M  10.7MB/s    in 0.4s    

2021-07-21 22:48:36 (10.7 MB/s) - ‘ml-100k.zip’ saved [4924029/4924029]

Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base

###ML-1M

In [6]:
!wget "http://files.grouplens.org/datasets/movielens/ml-1m.zip"
!unzip ml-1m.zip
!ls

--2021-07-21 22:48:37--  http://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  12.8MB/s    in 0.4s    

2021-07-21 22:48:38 (12.8 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         
ml-100k  ml-100k.zip  ml-1m  ml-1m.zip	sample_data


##Importing the libraries

In [25]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importing the dataset


In [27]:
# We won't be using this dataset.
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

## Preparing the training set and the test set


In [28]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype='int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype='int')

## Getting the number of users and movies


In [29]:
nb_users = int(max(max(training_set[:, 0], ), max(test_set[:, 0])))
nb_movies = int(max(max(training_set[:, 1], ), max(test_set[:, 1])))

## Converting the data into an array with users in lines and movies in columns


In [30]:
def convert(data):
  new_data = []
  for id_users in range(1, nb_users + 1):
    id_movies = data[:, 1] [data[:, 0] == id_users]
    id_ratings = data[:, 2] [data[:, 0] == id_users]
    ratings = np.zeros(nb_movies)
    ratings[id_movies - 1] = id_ratings
    new_data.append(list(ratings))
  return new_data
training_set = convert(training_set)
test_set = convert(test_set)

## Converting the data into Torch tensors


In [31]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Creating the architecture of the Neural Network


In [32]:
class SAE(nn.Module):
  def __init__(self, ):
    super(SAE, self).__init__()
    # full connection 1 (encoding)
    self.fc1 = nn.Linear(nb_movies, 50)
    # full connection 2 (encoding)
    self.fc2 = nn.Linear(50, 10)
    # full connection 3 (decoding)
    self.fc3 = nn.Linear(10, 50)
    # full connection 4 (decoding)
    self.fc4 = nn.Linear(50, nb_movies)
    self.activation = nn.Sigmoid()
  def forward(self, x):
    x = self.activation(self.fc1(x))
    x = self.activation(self.fc2(x))
    x = self.activation(self.fc3(x))
    x = self.fc4(x)
    return x


sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

## Training the SAE


In [33]:

nb_epoch = 300
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    # create batch with a single input vector
    input = Variable(training_set[id_user]).unsqueeze(0) # add a dimension to represent batch so pytorch accepts it
    target = input.clone()
    if torch.sum(target.data > 0) > 0:    
      output = sae(input)
      # optomize the code
      target.require_grad = False 
      output[target == 0] = 0
      # compute the loss
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward() # decides direction of update
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimizer.step() # decides the intensity of update
  print('epoch: ' + str(epoch) + ' loss: ' + str(train_loss/s))

epoch: 1 loss: tensor(1.5178)
epoch: 2 loss: tensor(1.0816)
epoch: 3 loss: tensor(1.0673)
epoch: 4 loss: tensor(1.0610)
epoch: 5 loss: tensor(1.0585)
epoch: 6 loss: tensor(1.0549)
epoch: 7 loss: tensor(1.0498)
epoch: 8 loss: tensor(1.0473)
epoch: 9 loss: tensor(1.0430)
epoch: 10 loss: tensor(1.0429)
epoch: 11 loss: tensor(1.0369)
epoch: 12 loss: tensor(1.0375)
epoch: 13 loss: tensor(1.0308)
epoch: 14 loss: tensor(1.0320)
epoch: 15 loss: tensor(1.0265)
epoch: 16 loss: tensor(1.0257)
epoch: 17 loss: tensor(1.0198)
epoch: 18 loss: tensor(1.0196)
epoch: 19 loss: tensor(1.0162)
epoch: 20 loss: tensor(1.0168)
epoch: 21 loss: tensor(1.0115)
epoch: 22 loss: tensor(1.0137)
epoch: 23 loss: tensor(1.0067)
epoch: 24 loss: tensor(1.0115)
epoch: 25 loss: tensor(1.0066)
epoch: 26 loss: tensor(1.0057)
epoch: 27 loss: tensor(1.0009)
epoch: 28 loss: tensor(1.0004)
epoch: 29 loss: tensor(0.9979)
epoch: 30 loss: tensor(1.0006)
epoch: 31 loss: tensor(0.9954)
epoch: 32 loss: tensor(0.9977)
epoch: 33 loss: t

## Testing the SAE


In [34]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9472)
